In [ ]:
import time
import pandas as pd # type: ignore
import pymysql # type: ignore
from selenium import webdriver # type: ignore
from selenium.webdriver.common.by import By # type: ignore
from selenium.webdriver.common.action_chains import ActionChains # type: ignore
from selenium.webdriver.support.ui import WebDriverWait # type: ignore
from selenium.webdriver.support import expected_conditions as EC # type: ignore


In [ ]:
# Function to collect route names and links from the current page
def extract_bus_routes(driver):
    route_names = []
    route_links = []
    routes = driver.find_elements(By.XPATH, "//a[@class='route']")
    for route in routes:
        route_names.append(route.get_attribute('title'))
        route_links.append(route.get_attribute('href'))
    return route_names, route_links


# Function to extract bus details
def collect_bus_details(driver, route_name, route_link):
    bus_details_list = []
    time.sleep(5)  # Consider replacing this with a wait for specific elements

    bus_items = driver.find_elements(By.XPATH, '//div[contains(@class, "bus-item")]')

    for bus_item in bus_items:
        bus_details = {
            "Bus Name": bus_item.find_element(By.XPATH, './/div[contains(@class, "travels")]').text.strip() or 'N/A',
            "Bus Type": bus_item.find_element(By.XPATH, './/div[contains(@class, "bus-type")]').text.strip() or 'N/A',
            "Departure Time": bus_item.find_element(By.XPATH,
                                                      './/div[contains(@class, "dp-time")]').text.strip() or 'N/A',
            "Arrival Time": bus_item.find_element(By.XPATH,
                                                    './/div[contains(@class, "bp-time")]').text.strip() or 'N/A',
            "Duration": bus_item.find_element(By.XPATH, './/div[contains(@class, "dur")]').text.strip() or 'N/A',
            "Price": bus_item.find_element(By.XPATH,
                                           './/div[contains(@class, "fare")]//span[contains(@class, "f-19 f-bold")]').text.strip() or 'N/A',
            "Star Rating": bus_item.find_element(By.XPATH,
                                                 './/div[contains(@class, "rating")]//span').text.strip() or 'N/A',
            "Seat Availability": bus_item.find_element(By.XPATH,
                                                       './/div[contains(@class, "seat-left")]').text.strip() or 'N/A',
            "Route Name": route_name,
            "Route Link": route_link
        }
        bus_details_list.append(bus_details)

    return bus_details_list


# Initialize the Chrome driver
driver = webdriver.Chrome()

try:
    # Open the desired URL
    driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
    wait = WebDriverWait(driver, 20)
 
    # Wait for the pagination container element
    pagination_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'DC_117_paginationTable')))

    all_route_names = []
    all_route_links = []

    # Iterate through each page
    for page in range(1, 6):  # Adjust the range based on your requirement
        xpath_expression = f'//div[contains(@class, "DC_117_pageTabs") and contains(text(), "{page}")]'
        
        page_button= pagination_container.find_element(By.XPATH, xpath_expression)

        actions = ActionChains(driver)
        actions.move_to_element(page_button).perform()
        page_button.click()
        time.sleep(10)  # Wait for the new page to load

        # Collect routes from the current page
        route_names, route_links = extract_bus_routes(driver)
        all_route_names.extend(route_names)
        all_route_links.extend(route_links)

    # Container for all bus details
    all_bus_details = []

    for route_link, route_name in zip(all_route_links, all_route_names):
        driver.get(route_link)
        driver.maximize_window()
        time.sleep(5)

        # Click all "View Buses" buttons
        try:
            view_buses_buttons = wait.until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, "//div[@class='button' and contains(text(),'View Buses')]"))
            )
            time.sleep(5)

            for button in reversed(view_buses_buttons):
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", button)
                    time.sleep(1)
                    button.click()
                    time.sleep(2)
                except Exception as e:
                    print(f"Error clicking button: {e}")
                    continue
        except Exception as e:
            print(f"Error during 'View Buses' button processing: {e}")

        
        # Scroll to the bottom of the page to ensure all buses are loaded
       
        scroll_pause_time = 2
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details after all content is loaded
        try:
            bus_details = collect_bus_details(driver, route_name, route_link)
            all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error extracting bus details for route {route_name}: {e}")

finally:
    # Close the browser
    driver.quit()

# Convert bus details to a DataFrame and remove duplicates
df = pd.DataFrame(all_bus_details)
# Display the DataFrame
print(df.head())

# Database connection details
user = 'root'
password = '123456789'
host = '127.0.0.1'
database = 'red'

# Connect to the MySQL database
connection = pymysql.connect(
    user=user, 
    password=password, 
    host=host, 
    database=database)
cursor = connection.cursor()

# Create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS Kerala_Bus_details(
    id INT AUTO_INCREMENT PRIMARY KEY,
    Bus_Name VARCHAR(100),
    Bus_Type VARCHAR(100),
    Departure_Time VARCHAR(100),
    Arrival_Time VARCHAR(100),
    Duration VARCHAR(100),
    Price FLOAT,
    Star_Rating FLOAT,
    Seat_Availability VARCHAR(100),
    Route_Name VARCHAR(500),
    Route_link VARCHAR(500)
)
"""
cursor.execute(create_table_query)

# Insert data into the table
for index, row in df.iterrows():
    try:
        insert_query = """
            INSERT  INTO  Kerala_Bus_details (
                Bus_Name, Bus_Type, Departure_Time, Arrival_Time, Duration, Price, Star_Rating, Seat_Availability, Route_Name, Route_link
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        price = float(row["Price"].replace(",", "")) if row["Price"] not in ['N/A', ''] else None
        star_rating = float(row["Star Rating"]) if row["Star Rating"] not in ['N/A', ''] else None

        cursor.execute(insert_query, (
            row["Bus Name"], row["Bus Type"], row["Departure Time"], row["Arrival Time"], row["Duration"],
            price, star_rating,
            row["Seat Availability"], row["Route Name"], row["Route Link"]
        ))
    except Exception as e:
        print(f"Error inserting row {index}: {e}")

# Commit and close the connection
try:
    connection.commit()
finally:
    cursor.close()
    connection.close()

Error extracting bus details for route Bangalore to Kozhikode: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class, "fare")]//span[contains(@class, "f-19 f-bold")]"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7102F6CB5+28821]
	(No symbol) [0x00007FF710263840]
	(No symbol) [0x00007FF71010578A]
	(No symbol) [0x00007FF7101591BE]
	(No symbol) [0x00007FF7101594AC]
	(No symbol) [0x00007FF71014C52C]
	(No symbol) [0x00007FF71017F33F]
	(No symbol) [0x00007FF71014C3F6]
	(No symbol) [0x00007FF71017F510]
	(No symbol) [0x00007FF71019F412]
	(No symbol) [0x00007FF71017F0A3]
	(No symbol) [0x00007FF71014A778]
	(No symbol) [0x00007FF71014B8E1]
	GetHandleVerifier [0x00007FF71062FCAD+3408013]
	GetHandleVerifier [0x00007FF71064741F+3504127]
	GetHandleVerifier [0x00007FF71063